In [1]:
import os, sys
import random
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from torchvision.utils import save_image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import transforms
from PIL import Image

from tqdm import tqdm

In [2]:
class RWTHAllImg(Dataset):
    """Class to load all images from RWTH dataset"""
    def __init__(self, globpattern, transform=None):
        self.imglobs = globpattern
        self.transform = transform
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = Image.open(self.imglobs[idx])
        if self.transform:
            img = self.transform(img)
        sample = {'image': img}
        return sample
    
    def __len__(self):
        return len(self.imglobs)

In [3]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        

        self.conv1 = nn.Conv2d(3, 6, padding=2, kernel_size=5) # out = b, 6, 
        
        self.maxpool1 = nn.MaxPool2d(4, stride=1, return_indices=True)
        self.conv2 = nn.Conv2d(6, 16, padding=2, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(2, stride=1, return_indices=True)
        
        self.unpool2 = nn.MaxUnpool2d(2, stride=1)
        self.unconv2 = nn.ConvTranspose2d(16, 6, padding=2, kernel_size=5)
        self.unpool1 = nn.MaxUnpool2d(4, stride=1)
        self.unconv1 = nn.ConvTranspose2d(6, 3, padding=2, kernel_size=5)
        
    def forward(self,x):
        # Encoder
        x = self.conv1(x)
        x = F.relu(x)
        x, indices1 = self.maxpool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x, indices2 = self.maxpool2(x)
        
        # Decoder
        x = self.unpool2(x, indices2)
        x = F.relu(x)
        x = self.unconv2(x)
        x = self.unpool1(x, indices1)
        x = F.relu(x)
        x = self.unconv1(x)
        #x = F.tanh(x)
        
        return x
    
    def encoder(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x, indices1 = self.maxpool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x, indices2 = self.maxpool2(x)
        return x, indices1, indices2

In [4]:
preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
traindir = "/home/reverie/datasets/phoenix2014-release/phoenix-2014-multisigner/features/fullFrame-210x260px/train"
train_glob = glob(os.path.join(traindir, "*/*/*.png"))
print(len(train_glob))
np.random.seed(3)
num_samples = 256 * 256
batch_size = 64
train_glob = np.random.choice(train_glob, num_samples, replace=False)
trainloader = torch.utils.data.DataLoader(RWTHAllImg(train_glob, transform=preprocess), batch_size=batch_size)

testdir = "/home/reverie/datasets/phoenix2014-release/phoenix-2014-multisigner/features/fullFrame-210x260px/test"
test_glob = glob(os.path.join(testdir, "*/*/*.png"))
print(len(test_glob))
test_glob  = np.random.choice(test_glob, 1000, replace=False)
testloader = torch.utils.data.DataLoader(RWTHAllImg(test_glob, transform=preprocess), batch_size=batch_size)

633621
89472


In [6]:
total_itr = num_samples // batch_size
total_itr

1024

In [7]:
model = Autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,
                             weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1, verbose = True, factor = 0.1, min_lr=1e-5, threshold=1e-3)
num_epochs = 5

In [8]:
print(model)

Autoencoder(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool1): MaxPool2d(kernel_size=4, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool2): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (unpool2): MaxUnpool2d(kernel_size=(2, 2), stride=(1, 1), padding=(0, 0))
  (unconv2): ConvTranspose2d(16, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (unpool1): MaxUnpool2d(kernel_size=(4, 4), stride=(1, 1), padding=(0, 0))
  (unconv1): ConvTranspose2d(6, 3, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)


In [9]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 3, 256, 256)
    return x

In [10]:
def save_checkpoint(model, optimizer, scheduler, epoch, loss, itr):
    chkpt = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict(),
        'epoch': epoch,
        'loss': loss
    }
    torch.save(chkpt, f'chkpt_{epoch}_{itr}.pth')

def load_checkpoint(path, model, optimizer, scheduler):
    if os.path.isfile(path):
        chpkt = torch.load(path)
        print(f"Loading checkpoint epoch:{chpkt['epoch']} and loss:{chpkt['loss']}")
        model.load_state_dict(chpkt['model'])
        optimizer.load_state_dict(chpkt['optimizer'])
        scheduler.load_state_dict(chpkt['scheduler'])
        return model, optimizer, scheduler
    else:
        print(f"{path} is not a valid file")

In [11]:
writer = SummaryWriter()

In [12]:
#model, optimizer, scheduler = load_checkpoint('./chkpt_4_1000.pth', model, optimizer, scheduler)
scheduler.threshold = 1e-5
for epoch in tqdm(range(num_epochs)):
    itr = 0
    for data in tqdm(trainloader):
        img = data['image']
        img = img.cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        writer.add_scalar(f'Loss/{epoch}', loss, itr)
        itr += 1
        if itr % 100 == 0:
            print(f"{itr}/{total_itr} iterations, loss:{loss}")
        if itr % 500 == 0:
            save_checkpoint(model, optimizer, scheduler, epoch, loss, itr)
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss))
    scheduler.step(loss)
    pic = to_img(output.cpu().data)
    #grid = torchvision.utils.make_grid(pic)
    #writer.add_image('images', grid)
    #writer.add_graph(model, pic.cuda())
    save_image(pic, './dc_img/image_{}.png'.format(epoch))

 10%|▉         | 100/1024 [04:17<54:57,  3.57s/it]

100/1024 iterations, loss:0.169610857963562



 20%|█▉        | 200/1024 [08:57<1:10:57,  5.17s/it]

200/1024 iterations, loss:0.11516422033309937



 29%|██▉       | 300/1024 [14:12<48:41,  4.04s/it]

...


 98%|█████████▊| 1000/1024 [09:43<00:13,  1.72it/s]

1000/1024 iterations, loss:0.06773126870393753



100%|██████████| 1024/1024 [09:57<00:00,  1.71it/s]


epoch [5/5], loss:0.0286


100%|██████████| 5/5 [1:26:32<00:00, 1038.48s/it]
